### IMPORT

In [ ]:
# --- 1. IMPORT LIBRERIE ---

import pandas as pd
import numpy as np

# GRAFICI
import matplotlib.pyplot as plt
import seaborn as sns

# MODULI SCIKIT-LEARN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# METRICHE
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Utility
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Impostazioni per il notebook
pd.set_option('display.max_columns', None)
print("Librerie importate con successo.")

### CARICAMENTO e ISPEZIONE DEI DATI

In [ ]:
# --- 2. CARICAMENTO DATI ---

file_path = 'movimento_fe.csv'

try:
    df = pd.read_csv(file_path)
    
    # Creiamo una copia su cui lavorare
    df_ml = df.copy()

    print(f"File '{file_path}' caricati con successo.")
    print(f"Dimensioni del dataset: {df_ml.shape[0]} righe e {df_ml.shape[1]} colonne.")

    print("Prime 5 righe del dataset:")
    display(df_ml.head())

    print("Informazioni sul dataset:")
    display(df_ml.info())

except FileNotFoundError:
    print(f"Errore: Il file '{file_path}' non è stato trovato.")

### CREAZIONE VARIABILI TARGET

In [ ]:
# --- 3. CREAZIONE VARIABILI TARGET (y) ---
print("Creazione colonne target...")

#1. target_binario: top player
df_ml['is_top_player'] = (df_ml['OVR'] <= 86).astype(int)

#2. Target binario sbilanciato: 5-star player
df_ml['is_5_star_player'] = (df_ml['Skill moves'] == 5).astype(int)

#3. Target multiclasse: Macro-Ruolo
#Definiamo una funzione per mappare i ruoli
POS_ATT = {'ST', 'CF', 'LW', 'RW', 'RF', 'LF'}
POS_CEN = {'CM', 'CDM', 'CAM', 'LM', 'RM'}
POS_DIF = {'CB', 'LB', 'RB', 'LWB', 'RWB'}

def mappa_macro_ruolo(pos):
    if pos in POS_ATT: return 'Attaccante'
    elif pos in POS_CEN: return 'Centrocampista'
    elif pos in POS_DIF: return 'Difensore'
    else: return np.nan

df_ml['Macro_Ruolo'] = df_ml['Position'].apply(mappa_macro_ruolo)

# Gestiamo eventuali ruoli non mappati
righe_prima = df_ml.shape[0]
df_ml = df_ml.dropna(subset=['Macro_Ruolo'])
righe_dopo = df_ml.shape[0]

if righe_prima > righe_dopo:
    print(f"Rimosse {righe_prima - righe_dopo} righe con 'Macro_Ruolo' nullo.")

print("Colonne target create con successo.")

# Verifica delle nuove colonne
print("\nVerifica delle nuove colonne target:")
display(df_ml[['is_top_player', 'is_5_star_player', 'Macro_Ruolo']].head())

### ENCODING FINALE delle FEATURE (X)

In [ ]:
# --- 4 . ENCODING FEATURES (X) ---
print("Inizio encoding delle feature...")

#1. 'Piede preferito' -> 0 (Destro), 1 (Sinistro)
# l'idea è vedere se il piede preferito è sinistro (true) o destro (false).
# astype(int) converte True/False in 1/0
df_ml['is_left_footed'] = (df_ml['Preferred Foot'] == 'Left').astype(int)

#2. 'Work Rate' -> Due colonne separate per attacco e difesa 
# NON SONO PRESENTI NEL DATASET DI PARTENZA MA LE LASCIO PER FUTURI USI
if 'Work rates' in df_ml.columns:
    try:
        # Dividiamo in due colonne
        wr_split = df_ml['Work rates'].str.split('/ ', expand=True)
        # Definiamo la mappatura ordinale
        work_rate_map = {'Low': 0, 'Medium': 1, 'High': 2}
        
        # Mappiamo e riempiamo i NaN con 'Medium' (1) come default
        df_ml['WR_Attack_num'] = wr_split[0].map(work_rate_map).fillna(1).astype(int)
        df_ml['WR_Defense_num'] = wr_split[1].map(work_rate_map).fillna(1).astype(int)
        
        print("Create 'WR_Attack_num' e 'WR_Defense_num'.")
    except Exception as e:
        print(f"Warning: Impossibile processare 'Work rates'. Errore: {e}")
else:
    print("Colonna 'Work rates' non trovata, encoding saltato.")

print("Encoding features completato.")

# Verifica
print("\nVerifica encoding:")
display(df_ml[['Name', 'Preferred foot', 'is_left_foot', 'Work rates', 'WR_Attack_num', 'WR_Defense_num']].head())

### PULIZIA FINALE

In [ ]:
# --- 5. PULIZIA FINALE ---
print("Inizio pulizia finale del dataset...")

colonne_da_rimuovere = [
    # testo ridondante
    'Alternative Positions',
    'play style',
    'Position',
    'Preferred Foot',
    
    # identifichiamo le colonne presenti che non useremo
    'Name',
    'Age',
    'Nation',
    'League',
    'Team'
]

colonne_esist_da_rimuovere = [col for col in colonne_da_rimuovere if col in df_ml.columns]

print(f"Rimozione delle colonne: {colonne_esist_da_rimuovere}")

# Eseguiamo la rimozione
df_ml = df_ml.drop(columns=colonne_esist_da_rimuovere)

print("Pulizia completata.")

### VERIFICA FINALE

In [ ]:
# --- 6. VERIFICA FINALE ---
print("Controllo finale del tipo di dato e valori nuilli...")
df_ml.info()

print("\nHead del DataFrame finale:")
display(df_ml.head())